This notebook handles the interaction with the Github API.
In a final step, the data must be serialized to be usable by the visualization-notebook.

The notebook must not necessarily be able to run on jupyterlite, though this would be optimal.